In [1]:
!pip install redis -q      # s’exécute une seule fois

import redis, json, pprint
r = redis.Redis(host='airbnb-redis', port=6379, decode_responses=True)
print("Ping ->", r.ping())

Ping -> True


In [2]:
from collections import Counter
prop_counter = Counter()

for key in r.scan_iter('movie:*'):
    prop = r.hget(key, 'genre') or 'unknown'
    prop_counter[prop] += 1

pprint.pprint(prop_counter)

Counter({'unknown': 92411})


In [3]:
count = 0
for key in r.scan_iter('movie:*'):
    if r.hget(key, 'release_year') == '2024-06-12' and r.hget(key, 'neighbourhood') == 'Paris':
        count += 1
print("Listings 12/06/2024 à Paris :", count)

Listings 12/06/2024 à Paris : 0


In [4]:
top = sorted(
    ((int(r.hget(k, 'votes')), k) for k in r.scan_iter('movie:*')),
    reverse=True
)[:5]

for votes, key in top:
    print(key, votes)

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [5]:
host_ids = {r.hget(k, 'host_id') for k in r.scan_iter('movie:*')}
print("Hôtes uniques :", len(host_ids))

Hôtes uniques : 69560


In [6]:
total = 0
instant = 0

for k in r.scan_iter('movie:*'):
    total += 1
    if r.hget(k, 'instant_bookable') == 'True':
        instant += 1

print("Instant bookable :", instant)
print("Proportion : {:.2%}".format(instant/total))

Instant bookable : 0
Proportion : 0.00%


In [7]:
from collections import Counter
host_counter = Counter()

for k in r.scan_iter('movie:*'):
    host_counter[r.hget(k, 'host_id')] += 1

big_hosts = {h:c for h,c in host_counter.items() if c > 100}
print(big_hosts)
print("Ils représentent {:.2%} des hôtes".format(len(big_hosts)/len(host_counter)))

{'440295601': 131, '50502817': 421, '374552379': 102, '33889201': 497, '512746089': 102, '499962530': 103, '314994947': 730, '26981054': 274, '51567288': 149, '21630783': 102, '50978178': 182, '506389460': 109, '28313443': 119, '125797498': 170, '436103373': 209, '1112584': 140, '335998296': 147, '517515174': 122, '460047164': 222, '564251645': 136, '528015349': 189, '99040006': 109, '7642792': 201, '24495283': 109}
Ils représentent 0.03% des hôtes


In [8]:
superhosts = {h for h in r.scan_iter('actor:*') if r.hget(h, 'about').lower().find('superhost') != -1}
print("Superhosts uniques :", len(superhosts))
print("Pourcentage : {:.2%}".format(len(superhosts)/len(host_counter)))

Superhosts uniques : 11
Pourcentage : 0.02%
